# Locate WesternUS pixels

Figure out pixel coordinates of Joan's pixels in Western US

In [1]:
%pylab notebook
from cetbtools.ease2conv import Ease2Transform
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import numpy as np

Populating the interactive namespace from numpy and matplotlib


Read a WesternUS cube to get projected and lat/lon coordinates.  
Any cube at a given resolution will do, they are all the same.
Use the UL corner of the cube to find out its corresponding offset from the 
full hemisphere grids.

In [4]:
def find_WesternUS_cube_offset(cubeType=None, verbose=False):
    if not cubeType:
        cubeType = '18H-GRD'
        
    cubeFile = "%s%s%s%s" % (
        "/work/PMESDR/CETB_v1.3/AQUA_AMSRE/N/cubes_WesternUS/",
        "CETB.cubefile.WesternUS.AQUA_AMSRE-",
        cubeType,
        "-RSS-v1.3.2003.TB.nc")
        
    f = Dataset(cubeFile, "r", "NETCDF4")   
    lats = f.variables['latitude'][:]
    lons = f.variables['longitude'][:]
    baseGPD = f.variables['crs'].long_name
    f.close()
    
    # find and return the baseGPD (row, col) offset for cubeUL(0, 0) location
    grid = Ease2Transform(baseGPD)
    row_offset, col_offset = grid.geographic_to_grid(lats[0,0], lons[0,0])

    if verbose:
        print("%10s offsets for cubeType=%s" % (baseGPD, cubeType))
        print("(Add these offsets to cube (row, col) to get row, col in full hemisphere)")
        print("offset row = %f, offset col = %f" % (row_offset, col_offset))
        
    return row_offset, col_offset    

In [5]:
types = ['18H-GRD', '18H-SIR', '36H-SIR']
for thisType in types:
    offset_row, offset_col = find_WesternUS_cube_offset(cubeType=thisType, verbose=True)

EASE2_N25km offsets for cubeType=18H-GRD
(Add these offsets to cube (row, col) to get row, col in full hemisphere)
offset row = 237.000000, offset col = 117.000000
EASE2_N6.25km offsets for cubeType=18H-SIR
(Add these offsets to cube (row, col) to get row, col in full hemisphere)
offset row = 948.000000, offset col = 468.000000
EASE2_N3.125km offsets for cubeType=36H-SIR
(Add these offsets to cube (row, col) to get row, col in full hemisphere)
offset row = 1896.000000, offset col = 936.000000


In [6]:
cubeFile = "/work/PMESDR/CETB_v1.3/AQUA_AMSRE/N/cubes_WesternUS/CETB.cubefile.WesternUS.AQUA_AMSRE-18H-GRD-RSS-v1.3.2003.TB.nc"
f = Dataset(cubeFile, "r", "NETCDF4")
x = f.variables['x'][:]
y = f.variables['y'][:]
lats = f.variables['latitude'][:]
lons = f.variables['longitude'][:]
print(f.variables['crs'].long_name)
f.close()

EASE2_N25km


Find out the subset UL corner geographic location and figure out 
which full hemisphere cell this corresponds to

In [9]:
# Figures out the WesternUS cube subset row,col coordinates at a given lat,lon
def grid_locations_of(lat, lon):
    gpds = ["EASE2_N3.125km", "EASE2_N6.25km", "EASE2_N25km"]
    types = ["36H-SIR", "18H-SIR", "18H-GRD"]
    
    print("Input lat, lon = %.6f, %.6f" % (lat, lon))
    rows = np.zeros((3))
    cols = np.zeros((3))
    for i, (thisGpd, thisType) in enumerate(zip(gpds, types)):
        grid = Ease2Transform(thisGpd)
        row, col = grid.geographic_to_grid(lat, lon)
        print("%15s         : row, col = %.6f, %.6f" % (thisGpd, row, col))
        
        # Get the cube offsets
        offset_row, offset_col = find_WesternUS_cube_offset(
            cubeType=thisType)
        WesternUSrow = row - offset_row
        WesternUScol = col - offset_col
        print("%15s(%s): row, col = %.6f, %.6f" % ("WesternUS", thisType, WesternUSrow, WesternUScol))
        rows[i] = WesternUSrow
        cols[i] = WesternUScol

    rows = (rows + 0.5).astype('int32')
    cols = (cols + 0.5).astype('int32')

    print(gpds)
    print("col=%d row=%d col=%d row=%d col=%d row=%d" % (
        cols[0], rows[0],
        cols[1], rows[1],
        cols[2], rows[2]))
    
    
    return (rows, cols)


In [10]:
# Columbia icefield
lat, lon = 52.14, -117.29
grid_locations_of(lat, lon)


Input lat, lon = 52.140000, -117.290000
 EASE2_N3.125km         : row, col = 2271.095756, 1700.233414
      WesternUS(36H-SIR): row, col = 375.095756, 764.233414
  EASE2_N6.25km         : row, col = 1135.297878, 849.866707
      WesternUS(18H-SIR): row, col = 187.297878, 381.866707
    EASE2_N25km         : row, col = 283.449469, 212.091677
      WesternUS(18H-GRD): row, col = 46.449469, 95.091677
['EASE2_N3.125km', 'EASE2_N6.25km', 'EASE2_N25km']
col=764 row=375 col=382 row=187 col=95 row=46


(array([375, 187,  46], dtype=int32), array([764, 382,  95], dtype=int32))

In [11]:
# SBSP Senator Beck, Colorado
lat, lon = 37.90688, -107.7262654
grid_locations_of(lat, lon)


Input lat, lon = 37.906880, -107.726265
 EASE2_N3.125km         : row, col = 2333.164037, 1170.308742
      WesternUS(36H-SIR): row, col = 437.164037, 234.308742
  EASE2_N6.25km         : row, col = 1166.332019, 584.904371
      WesternUS(18H-SIR): row, col = 218.332019, 116.904371
    EASE2_N25km         : row, col = 291.208005, 145.851093
      WesternUS(18H-GRD): row, col = 54.208005, 28.851093
['EASE2_N3.125km', 'EASE2_N6.25km', 'EASE2_N25km']
col=234 row=437 col=117 row=218 col=29 row=54


(array([437, 218,  54], dtype=int32), array([234, 117,  29], dtype=int32))

In [12]:
# OWL Old Wives Lake
lat, lon = 50.08639, -105.9480556
grid_locations_of(lat, lon)

Input lat, lon = 50.086390, -105.948056
 EASE2_N3.125km         : row, col = 2495.953563, 1537.323340
      WesternUS(36H-SIR): row, col = 599.953563, 601.323340
  EASE2_N6.25km         : row, col = 1247.726781, 768.411670
      WesternUS(18H-SIR): row, col = 299.726781, 300.411670
    EASE2_N25km         : row, col = 311.556695, 191.727917
      WesternUS(18H-GRD): row, col = 74.556695, 74.727918
['EASE2_N3.125km', 'EASE2_N6.25km', 'EASE2_N25km']
col=601 row=600 col=300 row=300 col=75 row=75


(array([600, 300,  75], dtype=int32), array([601, 300,  75], dtype=int32))